In [2]:
import openai 
import pandas as pd


In [3]:
dataset = pd.read_csv('supermarket_sales.csv')

In [32]:
dataset.head(5)
dataset['Rating'].mean()

np.float64(6.9727)

In [23]:
file = client.files.create(
	file=open('supermarket_sales.csv', 'rb'),
	purpose="assistants"
)

In [25]:
client = openai.Client()
assistant = client.beta.assistants.create(
    name="Analista Financeiro",
    instructions="Você é um analista financeiro de um supermercado. Você deve utlizar os dados\
     	.csv informados relativos as vendas do supermercado para realizar as suas análises",
    tools=[{"type": "code_interpreter"}],
    tool_resources={'code_interpreter': {'file_ids': [file.id]}},
    model="gpt-3.5-turbo",
)

In [26]:
thread = client.beta.threads.create()

In [27]:
# texto_mensagem = 'Qual é o rating médio das vendas do supermercado? O arquivo está no formato csv.'
message = client.beta.threads.messages.create(
	thread_id=thread.id,
 	role="user",
	content=texto_mensagem
)

In [28]:
run = client.beta.threads.runs.create(
	thread_id=thread.id,
	assistant_id=assistant.id,
	instructions="O nome do usuário é Adriano Soares e ele é um usuário Premium."
)

In [29]:
import time 

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
		thread_id=thread.id,
		run_id=run.id
	)

In [12]:
run.status

'completed'

In [30]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
		thread_id=thread.id
	)
    print(mensagens)

else:
    print('Error')

SyncCursorPage[Message](data=[Message(id='msg_WMpc4x7Azi09OodBTdtEcGyc', assistant_id='asst_IH5VrYjgUKT7Ph5qhEF07Hyy', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O rating médio das vendas do supermercado é aproximadamente 6.97. Se precisar de mais alguma informação ou análise, não hesite em me avisar!'), type='text')], created_at=1721415167, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_YoarKwERQTzK77ttg4rUEu88', status=None, thread_id='thread_lUXPrBuSs4GYkDW7s9hRHJ5t'), Message(id='msg_m8GWhQR12IdfriYffyLEcSaL', assistant_id='asst_IH5VrYjgUKT7Ph5qhEF07Hyy', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="O arquivo CSV fornecido contém várias colunas, incluindo 'Rating', que é a coluna que usaremos para calcular a média dos ratings de vendas do supermercado. Vou agora calcular o rating médio das vendas do supermercado com ba

In [31]:
print(mensagens.data[0].content[0].text.value)

O rating médio das vendas do supermercado é aproximadamente 6.97. Se precisar de mais alguma informação ou análise, não hesite em me avisar!


In [37]:
run_steps = client.beta.threads.runs.steps.list(
	thread_id=thread.id,
	run_id=run.id 
)

In [38]:
run_steps
run_steps.data[0]

RunStep(id='step_L7pRqS3p7vDIlFeRYJfhRlRR', assistant_id='asst_lMG6CYAx5Ov4yOS9giKjozCd', cancelled_at=None, completed_at=1721412818, created_at=1721412817, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_QihfbFFCBrAY70NN0CTuOZhu', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_3nOehgdr6Ud0f6dHRzIDCJUd'), type='message_creation'), thread_id='thread_C4hUXhRs88lEBt5vTNn7nc0z', type='message_creation', usage=Usage(completion_tokens=38, prompt_tokens=477, total_tokens=515), expires_at=None)

In [42]:
for step in run_steps.data:
    print('=== step: ', step.step_details.type)
    
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('----')
            print(tool_call.code_interpreter.input)
            print('----')
            print('Result')
            print(tool_call.code_interpreter.outputs)
    

=== step:  message_creation
=== step:  tool_calls
----
from math import comb

# Definindo os parâmetros
n = 1000
k = 150
p = 1/6

# Calculando a probabilidade
probabilidade = comb(n, k) * (p ** k) * ((1-p) ** (n-k))
probabilidade
----
Result
[]
=== step:  message_creation
